Inspired/based on https://github.com/rayidghani/magicloops and https://github.com/dssg/MLforPublicPolicy/blob/master/labs/2019/lab6_feature_generation_sol.ipynb

In [181]:
import pipeline_evictions as pipeline
import ml_loop_evictions as loop

import importlib
import datetime
import pandas as pd
import numpy as np

# Read data

In [182]:
importlib.reload(pipeline)

datafile = "data/tracts.csv"

#Read data, parsing year column to date type
data = pd.read_csv(datafile, parse_dates=['year'])

# Basic Cleaning

In [183]:
# Delete rows that do not have eviction-rate. We do not want to impute this value

data.dropna(subset=['eviction-rate'], inplace=True)
data['eviction-rate'].isnull().values.any()
data.reset_index(inplace=True, drop=True)

# Create outcome label

In [184]:
def get_eviction_rate(df, year,geoid):
    
  data_to_return = df.loc[(df['year'] == year) & (df['GEOID'] == geoid)]
  
  if(data_to_return.empty):
    return 0
  
  return data_to_return['eviction-rate'].iloc[0]

In [ ]:
importlib.reload(pipeline)

from dateutil.relativedelta import relativedelta

#Obtain eviction-rate cutoff for the top 10%, for each year
cutoff_10_percent={}
for year in range(2000,2017):
    year = pd.Timestamp(year,1,1)
    cutoff_10_percent[year]=data.loc[data['year'] == year]['eviction-rate'].quantile(.9)
    
top_10_eviction_rate_in_any_next_3_years_column = np.zeros(len(data))
print(top_10_eviction_rate_in_any_next_3_years_column.shape)

#Generate this feature now
# top_10_eviction_rate_last_year_column = np.zeros(len(data))

for index, row in data.iterrows():
  
  #Because the outcome will come from eviction-rate in next 3 years and we have data till 2016,
  #features data bust be from 2013 or before
  
  #We will also limit rows so that they all have one year before them (to calculate to_10_previous_year_feature)
  # & row['year']>=pd.Timestamp(2001,1,1)
  if(row['year']<=pd.Timestamp(2013,1,1)):    
    
    found_year_where_eviction_was_in_top_10_percent=0
    
    #Get eviction for the next 3 years
    for i in range(1,4):
      date_in_i_years = row['year'] + relativedelta(years=i)
      eviction_rate_in_i_years = get_eviction_rate(data, date_in_i_years,row['GEOID'])
    
      top_10_eviction_rate_in_i_years = 1 if eviction_rate_in_i_years>= cutoff_10_percent[date_in_i_years] else 0
      

      
      #If we found one year that meets requirement, we are done with looping
      if(top_10_eviction_rate_in_i_years==1):
        found_year_where_eviction_was_in_top_10_percent=1
        break

    top_10_eviction_rate_in_any_next_3_years_column[index]=found_year_where_eviction_was_in_top_10_percent

    
    #Generating feature now
#     date_last_years = row['year'] - relativedelta(years=i)
#     eviction_rate_last_year = get_eviction_rate(data, date_last_years,row['GEOID'])
#     top_10_eviction_rate_last_year = 1 if eviction_rate_last_year>= cutoff_10_percent[date_last_years] else 0

#     top_10_eviction_rate_last_year_column[index] = top_10_eviction_rate_last_year
    
    
data['top_10_percent_in_any_next_3_years'] = top_10_eviction_rate_in_any_next_3_years_column

label ='top_10_percent_in_any_next_3_years'

#Feature
# data['top_10_percent_last_year']=top_10_eviction_rate_last_year_column



In [186]:
#Debugging. In case we want to add 1 column for each future year with top 10%

      # eviction_rate_in_1_year = np.zeros(len(data))
# cutoff_in_1_year = np.zeros(len(data))

      
#       if(top_10_eviction_rate_in_i_years==1):
#         print(row['GEOID'])
#         print(row['year'])
#         print(date_in_i_years)
#         print(eviction_rate_in_i_years)
#         print(cutoff_10_percent[date_in_i_years])
#         print(top_10_eviction_rate_in_i_years)     

#       if(i==1):
#         eviction_rate_in_1_year[index]=eviction_rate_in_i_years
#         cutoff_in_1_year[index]=cutoff_10_percent[date_in_i_years]

# data['eviction_rate_in_1_year']='eviction_rate_in_1_year'
# data['cutoff_in_1_year']='eviction_rate_in_1_year'      
    

# Create temporal train and test sets

In [187]:
importlib.reload(pipeline)

#Create sets of train and test data, based on different split thresholds
#The split thresholds corresponds to the starting date of the testing data

#Splits according to https://docs.google.com/spreadsheets/d/1ipqsgThz7hdXXyyNpTuqa4J1inc088lop7lhFsAQ_r0/edit#gid=0
split_thresholds = [pd.Timestamp(i,1,1) for i in range (2004, 2014)]

#Indicating which is the column to be used for splitting training and test daata
date_column='year'

#Amount of data used for test set
test_window = relativedelta(years=4)

#Gap needed between training and test set
prediction_horizon = relativedelta(years=3)

#Generate train and test sets
train_test_sets= pipeline.create_temp_validation_train_and_testing_sets(
  data,
  date_column,
  label,
  split_thresholds,
  test_window,
  prediction_horizon)

In [188]:
for c in train_test_sets[5]['x_train'].columns:
  print(c)

GEOID
year
name
parent-location
population
poverty-rate
renter-occupied-households
pct-renter-occupied
median-gross-rent
median-household-income
median-property-value
rent-burden
pct-white
pct-af-am
pct-hispanic
pct-am-ind
pct-asian
pct-nh-pi
pct-multiple
pct-other
eviction-filings
evictions
eviction-rate
eviction-filing-rate
low-flag
imputed
subbed


# Imputation

In [189]:
importlib.reload(pipeline)

#Impute data on continuous columns for each training and test set

#--->PENDING
#In the meantime, imputing all float columns with mean
  
float_columns = [column for column in data.columns if data[column].dtype=='float']

#Do not consider GEOID column nor top_10_percent_in_any_next_3_years
float_columns.remove('name')
float_columns.remove('top_10_percent_in_any_next_3_years')


for train_test_set in train_test_sets:
  train_data = train_test_set['x_train']
  test_data = train_test_set['x_test']

  #fill na values with mean
  pipeline.impute_data(train_data, float_columns)
  pipeline.impute_data(test_data, float_columns)


# Create features

In [190]:
import feature_generation as fg

importlib.reload(pipeline)
importlib.reload(fg)

#We will have to generate features independently for each different train/test set
for train_test_set in train_test_sets:

  train_features = fg.create_features(train_test_set['x_train'])
  test_features = fg.create_features(train_test_set['x_test']) 
  
  #Alternative for just working with default features
  #train_features, test_features = pipeline.create_features(train_test_set)
  
  #Replace raw data in train_test_set with features generated
  train_test_set['x_train'] = train_features
  train_test_set['x_test'] = test_features

# Build Clasifiers and parameters generation

In [249]:
importlib.reload(pipeline)

#We define the specific models we want to run
models_to_run=['Baseline','DT','LR','RF','NB','BA','AB','GB']#,ET,KNN,SVM'

#Get all posible models and their different sets of parameters
models, parameters_grid = pipeline.get_models_and_parameters('small')

# Loop over models and different training/test sets

In [250]:
# # print(len(train_test_sets[0]['x_test']))
# # print(len(train_test_sets[0]['y_test']))
# train_test_sets[0]['y_test'].isnull().any()
# # df.isnull().values.any()

# # for column in train_test_sets[5]['x_test'].columns:
# #   print(column)
# #   
# train_test_sets[5]['x_train'].to_csv("columns.csv")


In [ ]:
importlib.reload(pipeline)
importlib.reload(loop)

import warnings

warnings.filterwarnings('ignore')

results = loop.iterate_over_models_and_training_test_sets(models_to_run, models, parameters_grid, train_test_sets)
results

2019-06-12 12:16:39.391763: Running Baseline with params: {} on train/test set 2004-01-01 00:00:00
2019-06-12 12:16:39.977862: Running DT with params: {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2} on train/test set 2004-01-01 00:00:00
2019-06-12 12:16:40.332184: Running DT with params: {'criterion': 'gini', 'max_depth': 50, 'min_samples_split': 2} on train/test set 2004-01-01 00:00:00
2019-06-12 12:16:41.045705: Running DT with params: {'criterion': 'gini', 'max_depth': 100, 'min_samples_split': 2} on train/test set 2004-01-01 00:00:00
2019-06-12 12:16:41.477932: Running LR with params: {'C': 0.1, 'penalty': 'l1'} on train/test set 2004-01-01 00:00:00
2019-06-12 12:16:42.383059: Running LR with params: {'C': 0.1, 'penalty': 'l2'} on train/test set 2004-01-01 00:00:00
2019-06-12 12:16:43.202484: Running LR with params: {'C': 10, 'penalty': 'l1'} on train/test set 2004-01-01 00:00:00
2019-06-12 12:17:04.143561: Running LR with params: {'C': 10, 'penalty': 'l2'} on train/t

2019-06-12 12:20:31.530818: Running RF with params: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100, 'n_jobs': -1} on train/test set 2006-01-01 00:00:00
2019-06-12 12:20:32.372730: Running RF with params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 10, 'n_jobs': -1} on train/test set 2006-01-01 00:00:00
2019-06-12 12:20:33.083332: Running RF with params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100, 'n_jobs': -1} on train/test set 2006-01-01 00:00:00
2019-06-12 12:20:34.102275: Running NB with params: {} on train/test set 2006-01-01 00:00:00
2019-06-12 12:20:34.535118: Running BA with params: {'n_estimators': 10} on train/test set 2006-01-01 00:00:00
2019-06-12 12:20:36.478362: Running BA with params: {'n_estimators': 100} on train/test set 2006-01-01 00:00:00
2019-06-12 12:20:54.649982: Running AB with params: {'algorithm': 'SAMME', 'n_estimators': 10} on train/test set 

2019-06-12 12:28:52.964157: Running GB with params: {'learning_rate': 0.001, 'n_estimators': 100} on train/test set 2008-01-01 00:00:00
2019-06-12 12:28:56.398076: Running GB with params: {'learning_rate': 0.1, 'n_estimators': 10} on train/test set 2008-01-01 00:00:00
2019-06-12 12:28:57.099110: Running GB with params: {'learning_rate': 0.1, 'n_estimators': 100} on train/test set 2008-01-01 00:00:00
2019-06-12 12:29:00.259359: Running ET with params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 100} on train/test set 2008-01-01 00:00:00
2019-06-12 12:29:00.907413: Running ET with params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 100} on train/test set 2008-01-01 00:00:00
2019-06-12 12:29:01.750266: Running ET with params: {'criterion': 'gini', 'max_depth': 50, 'n_estimators': 100} on train/test set 2008-01-01 00:00:00
2019-06-12 12:29:02.771695: Running ET with params: {'criterion': 'gini', 'max_depth': 50, 'n_estimators': 100} on train/test set 2008-01-01 00:00:00

# Observe best models for each train/test set, for different metrics

In [ ]:
results

In [ ]:
importlib.reload(pipeline)

#Lets obtain the best model for each train/test set, for each metric
metrics_to_display = ['p_at_5','p_at_10', 'auc-roc']

best_models_per_metric = {}

for metric in metrics_to_display:
    #indices of rows that have max value in specific metric for each train/test set
    idx = results.groupby(['test_set_start_date'])[metric].transform(max) == results[metric]

    #save table of best models at the specific metric
    best_models_per_metric[metric] = results[idx]

### Best models for Precision at 5%

In [ ]:
best_models_per_metric['p_at_5'].iloc[:, [0,2,3,4,11,12,13]]

### Best models for Precision at 10%

In [ ]:
best_models_per_metric['p_at_10'].iloc[:, [0,2,3,4,14,15,16]]

### Best models for AUC-ROC

In [ ]:
best_models_per_metric['auc-roc'].iloc[:, [0,2,3,4,26]]

### Plot of all model types performance at different train/test sets, for the different metrics

In [ ]:
importlib.reload(pipeline)

best_models_per_metric = {}

for metric in metrics_to_display:
    #For each model, find the set of parameters that work the best in each train/test set
    best_models = pipeline.get_best_models_of_each_type_for_each_train_test_set(models_to_run,results,'test_set_start_date', metric)
    pipeline.plot_models_in_time(models_to_run, best_models, metric)
    best_models_per_metric[metric]=best_models
   